# Zillow Neighborhood Dataset Analysis 



- It is strongly recommended you use the standard `pd`, `np`, `plt` and `sns` namespaces for these popular libraries


In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

## Load the Zillow dataset

In [ ]:
df = pd.read_csv("Neighborhood_Zhvi_AllHomes.csv")
df.head()

In [ ]:
df.shape

## How much are homes selling for in this dataset? 

In [ ]:
plt.hist(df["2019-07"]);

Yuck, that's a lot of overplotting. Let's fix it by rotating the ticks on the x-axis. 

In [ ]:
plt.hist(df["2019-07"]);
plt.xticks(rotation = 90);

The distribution of prices seems heavily skewed. 

How can we mitigate this so we can see the spread of the data better?

The log-transform is our friend for such heavily skewed distributions. 

(Aside: why?) 

In [ ]:
plt.hist(np.log10(df["2019-07"]));

It would be nice to compare how prices today compare to, say, 10 years ago. 

In [ ]:
sns.distplot(np.log10(df["2009-07"].dropna()), label ="July 2009");
sns.distplot(np.log10(df["2019-07"]), label = "July 2019");
plt.xlabel("Median Neighborhood Housing Prices (log scale)")
plt.ylabel("Density");
plt.legend();

## San Diego Analysis

Let's focus on understanding price changes in San Diego over time.

### Tidy Data
- Each variable you measure should be in one column.
- Each different observation of that variable should be in a different row.


In [ ]:
df.head()

- Our data is not tidy! 
Each price for each year is a different observation so should be in different row!

- `pd.melt` reshapes our data into the tidy format

In [ ]:
df_m = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 
                            'Metro', 'CountyName','SizeRank'])

df_m.head()

In [ ]:
df_m.shape

In [ ]:
df_sd = df_m[df_m.CountyName == 'San Diego County']

df_sd.head()

Lets explore how the median price of homes in each region has changed over time. 

In [ ]:
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .01);

- We get a good idea of the trends but over-plotting on the x-ticks makes it impossible to read. 

- We need to reduce the resolution of the x-ticks. 

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .01);

# create the text to use for labels for every 12 months
labels = []
pos = []
for i, item in enumerate(pd.unique(df_sd.variable)):
    if i % 12 ==0:
        pos.append(i)
        labels.append(item)

plt.xticks(pos, labels, rotation = 90);


This chart looks much better and shows the how prices or different regions are changing over time. It would be nice to know how a *specific* neighborhood's prices change. 

Let's get the list of all region names available to us in San Diego. 

In [ ]:
region_names = list(pd.unique(df_sd.RegionName))
len(region_names)

In [ ]:
plt.figure(figsize=(8,4))
df_region = df_sd[df_sd.RegionName == region_names[0]]
plt.plot(df_region.variable, df_region.value, '.')
labels = []
pos = []
for i, item in enumerate(pd.unique(df_region.variable)):
    if i % 12 ==0:
        pos.append(i)
        labels.append(item.split("-")[0])      
plt.xticks(pos, labels, rotation = 90);
plt.title(region_names[0]);

In [ ]:
plt.figure(figsize = (24,28))
num_cols_in_subplot = 9
for i, region_name in enumerate(sorted(region_names)):
    plt.subplot(len(region_names)//num_cols_in_subplot+1, num_cols_in_subplot, 
                i+1)
    # subset the data to just a specific region 
    df_region = df_sd[df_sd.RegionName == region_name]
    plt.plot(df_region.variable, df_region.value, linewidth = 2.2, 
             label = region_name)
    # make sure the vertical axis of each sub-plot has the same scale
    plt.ylim([np.min(df_sd.value), np.max(df_sd.value)])
    
    # only plot the y ticks that apppear on left
    if i%num_cols_in_subplot != 0:
        plt.yticks([])
    
    # only plot the x ticks that apppear on bottom
    if i < (len(region_names)//num_cols_in_subplot)*(num_cols_in_subplot):
        plt.xticks([]);  
    else:
        # lets just plot ticks for every other year
        # starting from an odd year
        plt.xticks(pos[1::2], labels[1::2], rotation = 90);
    
    plt.title(region_name, loc = "left")
    plt.subplots_adjust(wspace=0, hspace=.2)

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
df_region = df_sd[df_sd.RegionName == region_names[0]]
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .08);
plt.plot(df_region.variable, df_region.value, '.', color = "red")
plt.xticks(pos, labels, rotation = 90);
plt.title(region_names[0]);

plt.subplot(1,2,2)
df_region = df_sd[df_sd.RegionName == region_names[1]]
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .08);
plt.plot(df_region.variable, df_region.value, '.', color = "red")
plt.xticks(pos, labels, rotation = 90);
plt.title(region_names[1]);



In [ ]:
plt.figure(figsize = (24,28))

num_cols_in_subplot = 9

for i, region_name in enumerate(sorted(region_names)):
    plt.subplot(len(region_names)//num_cols_in_subplot+1, num_cols_in_subplot, 
                i+1)
    # plot all the regions but very lightly small alpha
    plt.plot(df_sd.variable, df_sd.value, '.', alpha = .01);
    # subset the data to just a specific region 
    df_region = df_sd[df_sd.RegionName == region_name]
    plt.plot(df_region.variable, df_region.value, linewidth = 2.2, 
             color = "red", label = region_name)
    
    # only plot the y ticks that apppear on left
    if i%num_cols_in_subplot != 0:
        plt.yticks([])
    
    # only plot the x ticks that apppear on bottom
    if i < (len(region_names)//num_cols_in_subplot)*(num_cols_in_subplot):
        plt.xticks([]);  
    else:
        # lets just plot ticks for every other year
        # starting from an odd year
        plt.xticks(pos[1::2], labels[1::2], rotation = 90);
    
    plt.title(region_name, loc = "left")
    plt.subplots_adjust(wspace=0, hspace=.2)

### Bubble Analysis

- The peak of the bubble occurred around 2005. 
- Instead of looking at prices at all time points in the data, lets compare prices at the height of the bubble with today. 
- We pick July 2005 as the "height" to correspond with the 14 years prior to the latest observations we have. 

In [ ]:
df_latest = df_sd[df_sd.variable == df_sd.variable.max()]
df_bubble = df_sd[df_sd.variable == '2005-07']
df_change = pd.concat([df_bubble, df_latest], ignore_index=True)

df_change.tail()

## Slope Graphs

In [ ]:
for i, region_name in enumerate(region_names):
    df_region = df_change[df_change.RegionName == region_name]
    plt.plot(df_region.variable, df_region.value, color = "k", alpha = .4)


We have too much over-plotting! 

Over-plotting solution: only plot the regions that have "large" changes. Might be also interesting to look at regions that went "negative" (have yet to recover from the crash). 

In [ ]:
for i, region_name in enumerate(region_names):
    df_region = df_change[df_change.RegionName == region_name]
    change = df_region.iloc[1,:]["value"] - df_region.iloc[0,:]["value"]
    percent_change = 100*change/df_region.iloc[0,:]["value"]
    if (change > 0) & (percent_change > 30):
        plt.plot(df_region.variable, df_region.value, color = "k", alpha = .4)
    elif (change < 0) & (percent_change < -5):
        plt.plot(df_region.variable, df_region.value, color = "r", alpha = .5)

Which neighborhoods are the biggest movers? Which neighborhoods did not recover?

In [ ]:
fig, ax = plt.subplots(figsize=(4, 12))
count = 0
for i, region_name in enumerate(region_names):
    df_region = df_change[df_change.RegionName == region_name]
    change = df_region.iloc[1,:]["value"] - df_region.iloc[0,:]["value"]
    percent_change = 100*change/df_region.iloc[0,:]["value"]
    if (change > 0) & (percent_change > 30):
        ax.plot(df_region.variable, df_region.value, color = "k", alpha = .4)
        ax.text(1.05, df_region.value.iloc[1], region_name)

    elif (change < 0) & (percent_change < -5):        
        ax.plot(df_region.variable, df_region.value, color = "r", alpha = .9)
        if count%2 == 0:
            ax.text(-.01, df_region.value.iloc[0], region_name, 
                    horizontalalignment = "right", verticalalignment = "top")
        else:
            ax.text(1.05, df_region.value.iloc[1], region_name,)
        count += 1

plt.title("Median Housing Prices");       

Percent change is often much more interesting. 

In [ ]:
# undo a pivot
df_sd_change = df_change.pivot(index = 'RegionName', columns = "variable", 
                               values = 'value')

df_sd_change.reset_index(inplace=True)

df_sd_change.head()

We will create a new variable `percent_change` that has the percent change between the latest entry in our data with the "peak" bubble.

In [ ]:
df_sd_change["percent_change"] = 100*(df_sd_change["2019-07"] - 
                                      df_sd_change["2005-07"])/df_sd_change["2005-07"]

In [ ]:
np.min(df_sd_change.percent_change), np.max(df_sd_change.percent_change)

In [ ]:
fig, ax = plt.subplots(figsize=(4,24), dpi= 80)
ax.hlines(y=np.arange(df_sd_change.shape[0]), alpha=0.7, 
          xmin=np.min(df_sd_change.percent_change) -1, 
          xmax=np.max(df_sd_change.percent_change) + 5, 
          linewidth=.5, linestyles='dashdot')
ax.scatter(y=np.arange(df_sd_change.shape[0]), 
           x=df_sd_change.percent_change, color='blue',s = 4, 
           alpha=0.7);


ax.set_yticks(np.arange(df_sd_change.shape[0]))
ax.set_yticklabels(df_sd_change.RegionName.str.title(), 
                   fontdict={'horizontalalignment': 'right',
                             'size': 8});
ax.set_ylim(-1, 123);

Alphabetically showing data is almost never useful. It is much more informative to display in *quantitative* sort

In [ ]:
df_sd_change.sort_values(["percent_change"], inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(4,24), dpi= 80)
ax.hlines(y=np.arange(df_sd_change.shape[0]), alpha=0.7, 
          xmin=np.min(df_sd_change.percent_change) -1, 
          xmax=np.max(df_sd_change.percent_change) + 5, 
          linewidth=.5, linestyles='dashdot')
ax.scatter(y=np.arange(df_sd_change.shape[0]), 
           x=df_sd_change.percent_change, color='blue',s = 4, 
           alpha=0.7);

ax.set_title('Median home price change from 2005-07 to 2019-07', 
             fontdict={'size':12})
ax.set_xlabel('Percent change')
ax.set_yticks(np.arange(df_sd_change.shape[0]))
ax.set_yticklabels(df_sd_change.RegionName.str.title(), 
                   fontdict={'horizontalalignment': 'right',
                             'size': 8});
ax.axvline(0)
ax.set_ylim(-1, 123);

## Price changes

In [ ]:
monthly_change = df_sd.groupby("RegionName")["value"].apply(lambda x: 
                                                 100*x.diff()/np.hstack((1,x[:-1])))
df_sd["monthly_change"] = monthly_change
df_sd.head()

In [ ]:
df_sd.tail()

In [ ]:
plt.figure(figsize=(7,8))
plt.subplot(2,1,1)
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .01);
plt.ylabel("Median monthly price");
plt.xticks([])
plt.subplot(2,1,2)
plt.plot(df_sd.variable, df_sd.monthly_change, '.', alpha = .01);
plt.xticks(pos, labels, rotation = 90);
plt.ylabel("Percently monthly price change")
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
df_region = df_sd[df_sd.RegionName == region_names[0]]
plt.plot(df_sd.variable, df_sd.monthly_change, '.', alpha = .08);
plt.plot(df_region.variable, df_region.monthly_change, '.', color = "red")
plt.xticks(pos, labels, rotation = 90);
plt.axhline(0, color = "k")
plt.ylabel("Percent monthly price change")
plt.title(region_names[0]);

plt.subplot(1,2,2)
df_region = df_sd[df_sd.RegionName == region_names[1]]
plt.plot(df_sd.variable, df_sd.monthly_change, '.', alpha = .08);
plt.plot(df_region.variable, df_region.monthly_change, '.', color = "red")
plt.xticks(pos, labels, rotation = 90);
plt.axhline(0, color = "k");
plt.title(region_names[1]);



In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
df_region = df_sd[df_sd.RegionName == region_names[0]]
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .08);
plt.plot(df_region.variable, df_region.value, '.', color = "red")
plt.xticks(pos, labels, rotation = 90);
plt.ylabel("Median monthly price")
plt.title(region_names[0]);

plt.subplot(1,2,2)
df_region = df_sd[df_sd.RegionName == region_names[1]]
plt.plot(df_sd.variable, df_sd.value, '.', alpha = .08);
plt.plot(df_region.variable, df_region.value, '.', color = "red")
plt.xticks(pos, labels, rotation = 90);
plt.title(region_names[1]);



In [ ]:
plt.figure(figsize = (24,28))
for i, region_name in enumerate(region_names):
    plt.subplot(14, 9, i+1)
    
    plt.plot(df_sd.variable, df_sd.monthly_change, '.', alpha = .01);
    plt.xticks([]);
    df_region = df_sd[df_sd.RegionName == region_name]
    plt.plot(df_region.variable, df_region.monthly_change, linewidth = 2.2, 
             color = "r", label = region_name)

    plt.xticks([]);
    if i%9 != 0:
        plt.yticks([])
    
    plt.title(region_name, loc = "left")
    plt.subplots_adjust(wspace=0, hspace=.2)

Let's compare the distribution of percent changes between two different states: early bubble state and present state we are at in the Real Estate market. 

In [ ]:
df_bubble_leadup = df_sd[(df_sd.variable>="2000-01") & (df_sd.variable<"2005-01")]

df_current_state = df_sd[(df_sd.variable>"2014-07") & (df_sd.variable<="2019-07")]

df_bubble_leadup.head()

Are the differences between the current state and the bubble state the same or different? Let's estimate some $p$-values for this.

In [ ]:
from scipy import stats

In [ ]:
p_vals = []
median_diffs = []
for region in region_names:
    df_bubble_leadup_region = df_bubble_leadup[df_bubble_leadup.RegionName == region].dropna()
    df_current_state_region = df_current_state[df_current_state.RegionName == region].dropna()
    if (df_bubble_leadup_region.shape[0] == 0) or (df_current_state_region.shape[0]==0):
        continue
    _, p_val = stats.ttest_ind(df_bubble_leadup_region.monthly_change, 
                               df_current_state_region.monthly_change)
    p_vals.append(p_val)
    
    median_diffs.append(np.median(np.abs(df_current_state_region.monthly_change)) - 
                        np.median(np.abs(df_bubble_leadup_region.monthly_change)))

## Volcano Plot


Volcano plots show us the relationship between the effect size and the statistical significance. The higher the effect size (more positive or more negative) and the higher on the graph (lower p-value) then the lesser chance of having False Positives. 

In [ ]:

plt.plot(median_diffs, -np.log10(p_vals), '.');
plt.axhline(-np.log10(.05), label = "statistically significant", color = "k")
plt.legend();
plt.xlabel("Effect size")
plt.ylabel("-log$_{10}p$");

## Box Plot


To be even more convincing, let's look at the distribution of price changes during the bubble and currently. 

In [ ]:
df_bubble_leadup["state"] = "bubble"
df_current_state["state"] = "current"


df_box_plot = pd.concat([df_bubble_leadup, df_current_state], axis = 0)

In [ ]:

ax = sns.boxplot(x='state', y='monthly_change', data=df_box_plot)


The box plot summarizes the data effectively, but the raw data is hidden. It would be nice to see some of the actual data itself. 

In [ ]:
np.random.seed(100)
ax = sns.boxplot(x='state', y='monthly_change', data=df_box_plot.sample(100))
np.random.seed(100)
ax = sns.swarmplot(x='state', y='monthly_change', data=df_box_plot.sample(100), color="grey")
